# Self Implemented Decision Tree Classifier for iris dataset of sklearn
### Made BY:- Arpnik Singh

## Considering all the columns to have discrete value

In [1]:
#imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score,precision_score,recall_score
from graphviz import Digraph

In [2]:
#load data
iris = datasets.load_iris()

In [3]:
#data frame formation
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']
y=iris.target

In [4]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [5]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
#df

In [6]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [7]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [8]:
#spliting dataframe into train and test sets
xtrain,xtest,ytrain,ytest=train_test_split(df,y,test_size=0.15)
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((127, 4), (23, 4), (127,), (23,))

In [9]:
#Implementation of node of Decision Tree
#col_name=column name to split upon
#isleaf is boolean value for detecting leaf node
#child nodes are stored in form of dictionary where key is the distinct value of column
#and value is pointer to a Tree Node 
#count stores total count of data points each node irrespective of their target class
class TreeNode:
    def __init__(self,column):
        self.col_name=column
        self.children=dict()
        self.isleaf=False
        self.count=0

In [10]:
#returns entropy of a node given the target labels of data points given as numpy array
def entropy(y):
    grp,cnt=np.unique(y,return_counts=True)
    total=sum(cnt)
    ent=0.000
    for i in range(grp.shape[0]):
        ent-=((cnt[i]/total)*np.log(cnt[i]/total))
    return ent;

In [11]:
#returns split information given a list containing number of data points in each child node after split
def split_info(t):
    tot=sum(t)
    info=0.0
    for i in t:
        info-=((i/tot)*np.log(i/tot))
    return info

In [12]:
#return gain ratio and entropy after split for a particular column
#col is name of the column
def current_column(x,y,col):
    grp,cnt=np.unique(x.loc[:,col],return_counts=True)
    #split information
    denom=split_info(cnt)
    #current entropy
    entc=entropy(y)
    split_ent=0.0
    tot=sum(cnt)
    for i in range(grp.shape[0]):
        temp=entropy(y[x.loc[:,col]==grp[i]])
        #weighted average of each entropy of child
        split_ent+=((cnt[i]*temp)/tot)
    #information gain
    infog=entc-split_ent
    #gain ratio
    gratio=infog/denom
    return gratio,split_ent
    

In [13]:
#building decision tree in depth first manner [hence the value of level]
#unsused is a list containing names of features left to split upon
def decision_tree(x,y,unused,count):
    #number of features left
    features=len(unused)
    print("Level",count)
    count+=1;
    #base case when pure class or no feature 
    if(np.unique(y).shape[0]<=1 or len(unused)==0):
        cls,num=np.unique(y,return_counts=True)
        for i in range(cls.shape[0]):
            print("Count of class",cls[i],'is',num[i])
        print("Current Entropy is",entropy(y))
        print("Reached a leaf Node")
        #print('unused=',len(unused))
        print()
        print()
        root=TreeNode('Predicted value:'+str(cls[np.argmax(num)]))
        root.isleaf=True
        root.predict=cls[np.argmax(num)]
        root.count=y.shape[0]
        return root;
    
    #bestcol is final decided column split upon
    #mgratio is maximum gain ratio
    #entc is current node entropy
    #mentc is entropy after split corresponding to maximum gain entropy
    bestcol=''
    mgratio=-999999999.000
    entc=entropy(y);
    mentc=-99999999.00
    #finding optimal feature/column to split upon
    for i in unused:
        cur_gratio,cur_split_entc=current_column(x,y,i)
        #print(i,cur_gratio)
        if(mgratio<cur_gratio):
            mgratio=cur_gratio
            bestcol=i;
            mentc=cur_split_entc
    print('Current Entropy:',entc)
    print('Feature used to split upon:',bestcol)
    print('Gain ratio after split:',mgratio)
    print('Net entropy after split is:',mentc)
    print()
    print()
    #removing split column
    unused.remove(bestcol)
    #make node
    root=TreeNode(bestcol)
    root.count=y.shape[0]
    cls,cnt=np.unique(y,return_counts=True)
    root.predict=cls[np.argmax(cnt)]
    for i in np.unique(x.loc[:,bestcol]):
        #creating data frame of child nodes
        tempx=x[x.loc[:,bestcol]==i].copy()
        tempy=y[x.loc[:,bestcol]==i].copy()
        del tempx[bestcol]
        #recursively creating tree
        temp_node=decision_tree(tempx,tempy,unused,count);
        root.children[i]=temp_node
    return root;
            

In [14]:
#building the decision tree and returning the head node
root=decision_tree(xtrain,ytrain,list(xtrain.columns),0)

Level 0
Current Entropy: 1.0981798219296024
Feature used to split upon: pl_labeled
Gain ratio after split: 0.6896548049778171
Net entropy after split is: 0.26485471572975466


Level 1
Count of class 0 is 41
Current Entropy is 0.0
Reached a leaf Node


Level 1
Count of class 1 is 7
Current Entropy is 0.0
Reached a leaf Node


Level 1
Current Entropy: 0.6228990536607193
Feature used to split upon: pw_labeled
Gain ratio after split: 0.4160394085353637
Net entropy after split is: 0.3100666269498895


Level 2
Count of class 1 is 4
Current Entropy is 0.0
Reached a leaf Node


Level 2
Current Entropy: 0.4293230219306162
Feature used to split upon: sl_labeled
Gain ratio after split: 0.12603900774175458
Net entropy after split is: 0.31491696825686727


Level 3
Count of class 2 is 1
Current Entropy is 0.0
Reached a leaf Node


Level 3
Count of class 1 is 12
Current Entropy is 0.0
Reached a leaf Node


Level 3
Current Entropy: 0.5117400734174093
Feature used to split upon: sw_labeled
Gain ratio a

In [15]:
#printing decision Tree
tree=Digraph('Decision-Tree')
if(root.isleaf):
    tree.attr('node',shape='circle')
else:
    tree.attr('node',shape='square')

temp=root.col_name
tree.node(temp)
def draw_tree(root):
    #global tree;
    if(root.isleaf):
        return;
    for i in root.children:
        tree.attr('node',shape='square')
        temp=root.children[i].col_name
        if(root.children[i].isleaf):
            tree.attr('node',shape='circle')
            temp='class '+root.children[i].col_name[-1]
        tree.node(temp)
        tree.edge(root.col_name,temp,label=i)
        draw_tree(root.children[i])
draw_tree(root)
tree.view()

'Decision-Tree.gv.pdf'

In [16]:
xtest.reset_index(inplace=True,drop=True)

In [17]:
#function to predict target value for 1 row/data point
#return 0,1,2
#0 for setosa
#1 for versicolor
#2 for virginica
def predict(x,root):
    #if(root==None):
    #    return None
    
    #base case when child node
    if(root.isleaf):
        return root.col_name[-1]
    
    #print(root.col_name,x.loc[root.col_name])
    #print(root.children)
    
    #if the data point is some how unique(which can be due to lack of data or some other reason) and we don't have an edge corresponding to that value
    #return value of that node
    if x.loc[root.col_name] not in root.children.keys():
        return root.predict
    
    #match the value of column and move further down in the tree
    return predict(x,root.children[x.loc[root.col_name]])

In [18]:
#function to predict target class for testing data
#returns a numpy array
def predictions(x,root):
    y=np.array([])
    for i in range(x.shape[0]):
        #print(i,x.iloc[i,:])
        y=np.append(y,[int(predict(x.iloc[i,:],root))])
        #print(i,y)
        #break
    return y;

In [19]:
ytest_predict=predictions(xtest,root)

In [20]:
#manual checking of values
for i in range(ytest.shape[0]):
    print(ytest[i],int(ytest_predict[i]))

0 0
1 1
2 2
2 2
1 1
2 2
1 1
1 1
0 0
2 2
0 0
1 1
0 0
2 2
0 0
2 1
2 2
0 0
0 0
0 0
2 1
1 1
0 0


In [21]:
print('Confusion Matrix')
print(confusion_matrix(ytest,ytest_predict))
print()
print()
print("F1 Score of each class (in this order 'setosa', 'versicolor', 'virginica')")
print(f1_score(ytest,ytest_predict,average=None))
print()
print("Precision for each class (in this order 'setosa', 'versicolor', 'virginica')")
print(precision_score(ytest,ytest_predict,average=None))
print()
print("Recall of each class (in this order 'setosa', 'versicolor', 'virginica')")
print(recall_score(ytest,ytest_predict,average=None))

Confusion Matrix
[[9 0 0]
 [0 6 0]
 [0 2 6]]


F1 Score of each class (in this order 'setosa', 'versicolor', 'virginica')
[1.         0.85714286 0.85714286]

Precision for each class (in this order 'setosa', 'versicolor', 'virginica')
[1.   0.75 1.  ]

Recall of each class (in this order 'setosa', 'versicolor', 'virginica')
[1.   1.   0.75]


## Considering all the columns to have continous values

In [22]:
data=datasets.load_iris()
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [23]:
X=data.data
Y=data.target
col=data.feature_names
target_class=data.target_names
print(X.shape,Y.shape)
print(data.DESCR)
X=pd.DataFrame(X,columns=col)

(150, 4) (150,)
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= =

In [24]:
#spliting data to train and test set
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.1)
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((135, 4), (15, 4), (135,), (15,))

## Decision Tree is build assuming only binary cuts are possible

In [25]:
#node of decision tree
#cut is the value at which continous vale mark is made
class TreeNode:
    def __init__(self,column,mark):
        self.col_name=column
        self.left=None
        self.right=None
        self.cut=mark

In [26]:
#finding entropy of a node 
def entropy(y):
    grp,cnt=np.unique(y,return_counts=True)
    total=sum(cnt)
    ent=0.000
    for i in range(grp.shape[0]):
        ent-=((cnt[i]/total)*np.log(cnt[i]/total))
    return ent;

In [27]:
#split info of child nodes of left and right sub tree
def split_info(t1,t2):
    tot=t1+t2
    info=-((t1/tot)*np.log(t1/tot))
    info-=((t2/tot)*np.log(t2/tot))
    return info

In [28]:
#finding optimal cut value for a particular column
#returns max gain ratio,cut value and entropy after split (at max gain ratio) for a particular column
def column_seperator(x,y,col):
    #sorting data frame according to column values
    indices=np.argsort(x.loc[:,col])
    x=x.iloc[indices,:]
    y=y[indices]
    #current entropy
    entc=entropy(y)
    x.reset_index(drop=True,inplace=True)
    max_gratio=-999999999
    max_ent=-999999
    sep=-99999999
    #traversing at every mid point of the column
    for i in range(1,x.shape[0]):
        mark=(x.loc[i,col]+x.loc[i-1,col])/2
        indices=(x.loc[:,col]>mark)
        ent1=entropy(y[indices])
        tot1=y[indices].shape[0]
        indices=(x.loc[:,col]<=mark)
        ent2=entropy(y[indices])
        tot2=y[indices].shape[0]
        total=tot1+tot2
        #entropy after split
        finalent=(tot1*ent1+tot2*ent2)/total
        #information gain
        infog=entc-finalent
        spt_info=split_info(tot1,tot2)
        #gain ratio
        gratio=infog/spt_info
        if(max_gratio<gratio):
            max_gratio=gratio
            sep=mark
            max_ent=finalent
    return max_gratio,sep,max_ent
        

In [29]:
count=0
#build Decision tree
def decision_tree(x,y):
    global count
    #number of features left
    features=x.shape[1]
    print("Level",count)
    count+=1;
    #base case when no feature or pure class
    if(np.unique(y).shape[0]<=1 or x.shape[1]==0):
        cls,num=np.unique(y,return_counts=True)
        for i in range(cls.shape[0]):
            print("Count of class",cls[i],'is',num[i])
        print("Current Entropy is",entropy(y))
        print("Reached a leaf Node")
        print()
        print()
        root=TreeNode('Child Node predicted value:'+str(cls[np.argmax(num)]),0.0)
        root.left=root.right=None
        return root;
    
    #splitcol is column decided to split upon
    #mgratio is maximum gain ratio
    #entc is cureent entropy
    #cut is mark at which continous value of column is breaked 
    splitcol=''
    cut=-9999999.000
    mgratio=-999999999.000
    entc=entropy(y);
    mentc=-99999999.00
    #finding optimal feature/column to split upon
    for i in x.columns:
        cur_gratio,mark,cur_entc=column_seperator(x,y,i)
        if(mgratio<cur_gratio):
            mgratio=cur_gratio
            splitcol=i;
            cut=mark;
            mentc=cur_entc
    print('Current Entropy:',entc)
    print('Gain ratio:',mgratio)
    print('Column to split upon:',splitcol)
    print('cut for this column at',cut)
    print()
    print()
    x1=x[x.loc[:,splitcol]<=cut]
    x2=x[x.loc[:,splitcol]>cut]
    
    #Making tree
    root=TreeNode(splitcol,cut)
    root.left=decision_tree(x1,y[x.loc[:,splitcol]<=cut])
    root.right=decision_tree(x2,y[x.loc[:,splitcol]>cut])
    return root;
    
            

In [30]:
#start node of decision tree
root=decision_tree(xtrain,ytrain)

Level 0


/home/yay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/home/yay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


Current Entropy: 1.0981146787953189
Gain ratio: 0.9999999999999997
Column to split upon: petal length (cm)
cut for this column at 1.9


Level 1
Count of class 0 is 46
Current Entropy is 0.0
Reached a leaf Node


Level 2
Current Entropy: 0.6925789628408865
Gain ratio: 0.7818745691179838
Column to split upon: petal width (cm)
cut for this column at 1.7


Level 3
Current Entropy: 0.2787693717685874
Gain ratio: 0.5411340426841159
Column to split upon: sepal length (cm)
cut for this column at 7.1


Level 4
Current Entropy: 0.23032354087587759
Gain ratio: 0.268351493307326
Column to split upon: petal length (cm)
cut for this column at 4.9


Level 5
Current Entropy: 0.10656595882801999
Gain ratio: 1.0
Column to split upon: petal width (cm)
cut for this column at 1.6


Level 6
Count of class 1 is 44
Current Entropy is 0.0
Reached a leaf Node


Level 7
Count of class 2 is 1
Current Entropy is 0.0
Reached a leaf Node


Level 8
Current Entropy: 0.6931471805599453
Gain ratio: 1.0
Column to split u

In [31]:
xtest.reset_index(inplace=True,drop=True)


In [32]:
#function to predict target value for 1 row/data point
def predict(x,root):
    #base case when child node
    #print(x.loc[root.col_name],root.cut)
    if(root.col_name.startswith('Child')):
        return root.col_name[-1]
    
    if(x.loc[root.col_name]>root.cut):
        return predict(x,root.right)
    else:
        return predict(x,root.left)
        

In [33]:
#function to predict targte values for test set
def predictions(x,root):
    y=np.array([])
    for i in range(x.shape[0]):
        y=np.append(y,[int(predict(x.iloc[i,:],root))])
        #break
    return y;

In [34]:
ytest_predict=predictions(xtest,root)
ytest_predict=ytest_predict.astype('int64')

In [35]:
#manual checking
for i in range(ytest.shape[0]):
    print(ytest_predict[i],ytest[i])

0 0
1 1
0 0
0 0
2 2
1 1
1 1
2 2
2 1
2 2
2 2
2 2
0 0
2 2
2 2


In [36]:
print('Confusion Matrix')
print(confusion_matrix(ytest,ytest_predict))
print()
print()
print("F1 Score of each class (in this order 'setosa', 'versicolor', 'virginica')")
print(f1_score(ytest,ytest_predict,average=None))
print()
print("Precision for each class (in this order 'setosa', 'versicolor', 'virginica')")
print(precision_score(ytest,ytest_predict,average=None))
print()
print("Recall of each class (in this order 'setosa', 'versicolor', 'virginica')")
print(recall_score(ytest,ytest_predict,average=None))

Confusion Matrix
[[4 0 0]
 [0 3 1]
 [0 0 7]]


F1 Score of each class (in this order 'setosa', 'versicolor', 'virginica')
[1.         0.85714286 0.93333333]

Precision for each class (in this order 'setosa', 'versicolor', 'virginica')
[1.    1.    0.875]

Recall of each class (in this order 'setosa', 'versicolor', 'virginica')
[1.   0.75 1.  ]


In [37]:
#verifying the results with sklearns classifier
alg2=DecisionTreeClassifier()
alg2.fit(xtrain,ytrain)
ypred2=alg2.predict(xtest)
print(confusion_matrix(ytest,ypred2))

[[4 0 0]
 [0 3 1]
 [0 0 7]]
